In [2]:
import numpy as np 
import mathplotlib.pyplot as plt 
import pandas as pd 
import pandas_datareader as web 
import datetime as dt 

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Dropout, LSTM




ModuleNotFoundError: No module named 'mathplotlib'

# Load Data

In [ ]:
company = 'FB'

start = dt.datetime(2012,1,1)
end = dt.datetime(2020,1,1)


data = web.DataReader(company, 'yahoo', start, end)

### Prepare Data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['close'].values.reshape(-1,1))

prediction_days = 60


x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x,0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train[0], x_train.shape[1], 1))

## Built  The   Model

In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))  #Prediction of the next closing values

model.compile(optimizer='adam'), loss='mean_squared_error'
model.fit(x_train, y_train, epochs=25, batch_size = 32)

"" Test the model accuracy on existing Data ""

In [ ]:
test_start = dt.datetime(2020, 1, 1)
test_end = dt.datetime.now()

test_data =web.DataReader(company, 'yahoo', test_start, test_end)
actual_price = test_data['close'].values

total_dataset = pd.concat((data['close'], test_start['close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)